In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [24]:
df=pd.read_csv('Social_Network_Ads.csv')
df=pd.get_dummies(df,dtype=int,drop_first=True)
df

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [25]:
df.isnull().values.any()

np.False_

In [26]:
indep=df[['User ID','Age','EstimatedSalary','Gender_Male']]
dep=df['Purchased']

In [27]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=1/3,random_state=3)

In [39]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

SUPPORT_VECTOR_MACHINE

In [40]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
param_grid={'kernel':['rbf','sigmoid','linear'],
              'C':[1,10],'gamma':['auto','scale']}
grid=GridSearchCV(SVC(probability=True),param_grid,refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [1, 10], 'gamma': ['auto', 'scale'],
                         'kernel': ['rbf', 'sigmoid', 'linear']},
             scoring='f1_weighted', verbose=3)

In [41]:
re=grid.cv_results_
y_pred=grid.predict(x_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)

from sklearn.metrics import classification_report
cr=classification_report(y_test,y_pred)
print(cr)

[[79  9]
 [ 4 42]]
              precision    recall  f1-score   support

           0       0.95      0.90      0.92        88
           1       0.82      0.91      0.87        46

    accuracy                           0.90       134
   macro avg       0.89      0.91      0.89       134
weighted avg       0.91      0.90      0.90       134



In [42]:
from sklearn.metrics import f1_score
fs=f1_score(y_test,y_pred,average='weighted')
print("The f1_macro value for the best parameters {}:".format(grid.best_params_),fs)

from sklearn.metrics import roc_auc_score
roc_auc_score=(y_test,grid.predict_proba(x_test)[:1])
roc_auc_score

The f1_macro value for the best parameters {'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}: 0.9040671124392506


(376    0
 16     1
 365    1
 82     0
 107    0
       ..
 251    0
 201    0
 386    1
 68     0
 193    0
 Name: Purchased, Length: 134, dtype: int64,
 array([[0.43814179, 0.56185821]]))

DECISION_TREE

In [43]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
param_grid={'criterion':['log_loss', 'entropy', 'gini'],
              'max_features': [None,'sqrt','log2'],
              'max_depth': [None, 10, 20],  # Use max_depth instead of max_features
    'min_samples_split': [2, 10]}
grid=GridSearchCV(DecisionTreeClassifier(),param_grid,refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')
grid.fit(x_train,y_train)



Fitting 5 folds for each of 54 candidates, totalling 270 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['log_loss', 'entropy', 'gini'],
                         'max_depth': [None, 10, 20],
                         'max_features': [None, 'sqrt', 'log2'],
                         'min_samples_split': [2, 10]},
             scoring='f1_weighted', verbose=3)

In [44]:
re=grid.cv_results_
y_pred=grid.predict(x_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)

from sklearn.metrics import classification_report
cr=classification_report(y_test,y_pred)
print(cr)

from sklearn.metrics import f1_score
fs=f1_score(y_test,y_pred,average='weighted')
print("The f1_macro value for the best parameters {}:".format(grid.best_params_),fs)

from sklearn.metrics import roc_auc_score
roc_auc_score=(y_test,grid.predict_proba(x_test)[:1])
roc_auc_score

[[80  8]
 [ 8 38]]
              precision    recall  f1-score   support

           0       0.91      0.91      0.91        88
           1       0.83      0.83      0.83        46

    accuracy                           0.88       134
   macro avg       0.87      0.87      0.87       134
weighted avg       0.88      0.88      0.88       134

The f1_macro value for the best parameters {'criterion': 'log_loss', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 10}: 0.8805970149253731


(376    0
 16     1
 365    1
 82     0
 107    0
       ..
 251    0
 201    0
 386    1
 68     0
 193    0
 Name: Purchased, Length: 134, dtype: int64,
 array([[0.33333333, 0.66666667]]))

RANDOM_FOREST

In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
param_grid={'criterion':['log_loss', 'entropy', 'gini'],
              'max_features': [None,'sqrt','log2'],
              'n_estimators':[10,100]} 
grid=GridSearchCV(RandomForestClassifier(),param_grid,refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['log_loss', 'entropy', 'gini'],
                         'max_features': [None, 'sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='f1_weighted', verbose=3)

In [46]:
re=grid.cv_results_
y_pred=grid.predict(x_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)

from sklearn.metrics import classification_report
cr=classification_report(y_test,y_pred)
print(cr)

from sklearn.metrics import f1_score
fs=f1_score(y_test,y_pred,average='weighted')
print("The f1_macro value for the best parameters {}:".format(grid.best_params_),fs)

from sklearn.metrics import roc_auc_score
roc_auc_score=(y_test,grid.predict_proba(x_test)[:1])
roc_auc_score

[[79  9]
 [ 5 41]]
              precision    recall  f1-score   support

           0       0.94      0.90      0.92        88
           1       0.82      0.89      0.85        46

    accuracy                           0.90       134
   macro avg       0.88      0.89      0.89       134
weighted avg       0.90      0.90      0.90       134

The f1_macro value for the best parameters {'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 100}: 0.8964841490223302


(376    0
 16     1
 365    1
 82     0
 107    0
       ..
 251    0
 201    0
 386    1
 68     0
 193    0
 Name: Purchased, Length: 134, dtype: int64,
 array([[0.27, 0.73]]))

LogisticRegression

In [47]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': [0.1, 1, 10, 100],
    'solver': ['saga'],
    'max_iter': [50,100],
    'l1_ratio': [0.1, 0.5, 0.7, 1]
}
grid=GridSearchCV(LogisticRegression(),param_grid,refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


C:\Users\Hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1197: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100], 'l1_ratio': [0.1, 0.5, 0.7, 1],
                         'max_iter': [50, 100],
                         'penalty': ['l1', 'l2', 'elasticnet'],
                         'solver': ['saga']},
             scoring='f1_weighted', verbose=3)

In [48]:
re=grid.cv_results_
y_pred=grid.predict(x_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)

from sklearn.metrics import classification_report
cr=classification_report(y_test,y_pred)
print(cr)

from sklearn.metrics import f1_score
fs=f1_score(y_test,y_pred,average='weighted')
print("The f1_macro value for the best parameters {}:".format(grid.best_params_),fs)

from sklearn.metrics import roc_auc_score
roc_auc_score=(y_test,grid.predict_proba(x_test)[:1])
roc_auc_score

[[83  5]
 [18 28]]
              precision    recall  f1-score   support

           0       0.82      0.94      0.88        88
           1       0.85      0.61      0.71        46

    accuracy                           0.83       134
   macro avg       0.84      0.78      0.79       134
weighted avg       0.83      0.83      0.82       134

The f1_macro value for the best parameters {'C': 0.1, 'l1_ratio': 0.1, 'max_iter': 50, 'penalty': 'l1', 'solver': 'saga'}: 0.8201388076695086


(376    0
 16     1
 365    1
 82     0
 107    0
       ..
 251    0
 201    0
 386    1
 68     0
 193    0
 Name: Purchased, Length: 134, dtype: int64,
 array([[0.38631579, 0.61368421]]))

KNN


In [49]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
param_grid = {'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
   }
grid=GridSearchCV(KNeighborsClassifier(),param_grid,refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [3, 5, 7, 9], 'p': [1, 2],
                         'weights': ['uniform', 'distance']},
             scoring='f1_weighted', verbose=3)

In [50]:
re=grid.cv_results_
y_pred=grid.predict(x_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)

from sklearn.metrics import classification_report
cr=classification_report(y_test,y_pred)
print(cr)

from sklearn.metrics import f1_score
fs=f1_score(y_test,y_pred,average='weighted')
print("The f1_macro value for the best parameters {}:".format(grid.best_params_),fs)

from sklearn.metrics import roc_auc_score
roc_auc_score=(y_test,grid.predict_proba(x_test)[:1])
roc_auc_score

[[82  6]
 [ 6 40]]
              precision    recall  f1-score   support

           0       0.93      0.93      0.93        88
           1       0.87      0.87      0.87        46

    accuracy                           0.91       134
   macro avg       0.90      0.90      0.90       134
weighted avg       0.91      0.91      0.91       134

The f1_macro value for the best parameters {'n_neighbors': 9, 'p': 1, 'weights': 'distance'}: 0.9104477611940298


(376    0
 16     1
 365    1
 82     0
 107    0
       ..
 251    0
 201    0
 386    1
 68     0
 193    0
 Name: Purchased, Length: 134, dtype: int64,
 array([[0.65207435, 0.34792565]]))

Naive_bayes

In [51]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
param_grid = {'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5] 
   }
grid=GridSearchCV(GaussianNB(),param_grid,refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


GridSearchCV(estimator=GaussianNB(), n_jobs=-1,
             param_grid={'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05]},
             scoring='f1_weighted', verbose=3)

In [52]:
re=grid.cv_results_
y_pred=grid.predict(x_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)

from sklearn.metrics import classification_report
cr=classification_report(y_test,y_pred)
print(cr)

from sklearn.metrics import f1_score
fs=f1_score(y_test,y_pred,average='weighted')
print("The f1_macro value for the best parameters {}:".format(grid.best_params_),fs)

from sklearn.metrics import roc_auc_score
roc_auc_score=(y_test,grid.predict_proba(x_test)[:1])
roc_auc_score

[[81  7]
 [ 8 38]]
              precision    recall  f1-score   support

           0       0.91      0.92      0.92        88
           1       0.84      0.83      0.84        46

    accuracy                           0.89       134
   macro avg       0.88      0.87      0.88       134
weighted avg       0.89      0.89      0.89       134

The f1_macro value for the best parameters {'var_smoothing': 1e-09}: 0.8877608604398385


(376    0
 16     1
 365    1
 82     0
 107    0
       ..
 251    0
 201    0
 386    1
 68     0
 193    0
 Name: Purchased, Length: 134, dtype: int64,
 array([[0.39378564, 0.60621436]]))

In [53]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import BernoulliNB
param_grid = { 'alpha': [0.5, 1.0, 1.5, 2.0], 
    'binarize': [0.0, 0.5, 1.0]  
   }
grid=GridSearchCV(BernoulliNB(),param_grid,refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=BernoulliNB(), n_jobs=-1,
             param_grid={'alpha': [0.5, 1.0, 1.5, 2.0],
                         'binarize': [0.0, 0.5, 1.0]},
             scoring='f1_weighted', verbose=3)

In [54]:
re=grid.cv_results_
y_pred=grid.predict(x_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)

from sklearn.metrics import classification_report
cr=classification_report(y_test,y_pred)
print(cr)

from sklearn.metrics import f1_score
fs=f1_score(y_test,y_pred,average='weighted')
print("The f1_macro value for the best parameters {}:".format(grid.best_params_),fs)

from sklearn.metrics import roc_auc_score
roc_auc_score=(y_test,grid.predict_proba(x_test)[:1])
roc_auc_score

[[84  4]
 [15 31]]
              precision    recall  f1-score   support

           0       0.85      0.95      0.90        88
           1       0.89      0.67      0.77        46

    accuracy                           0.86       134
   macro avg       0.87      0.81      0.83       134
weighted avg       0.86      0.86      0.85       134

The f1_macro value for the best parameters {'alpha': 0.5, 'binarize': 1.0}: 0.852751493079266


(376    0
 16     1
 365    1
 82     0
 107    0
       ..
 251    0
 201    0
 386    1
 68     0
 193    0
 Name: Purchased, Length: 134, dtype: int64,
 array([[0.82754112, 0.17245888]]))

In [34]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
param_grid = { 'alpha': [0.5, 1.0, 1.5, 2.0] 
   }
grid=GridSearchCV(MultinomialNB(),param_grid,refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(estimator=MultinomialNB(), n_jobs=-1,
             param_grid={'alpha': [0.5, 1.0, 1.5, 2.0]}, scoring='f1_weighted',
             verbose=3)

In [35]:
re=grid.cv_results_
y_pred=grid.predict(x_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)

from sklearn.metrics import classification_report
cr=classification_report(y_test,y_pred)
print(cr)

from sklearn.metrics import f1_score
fs=f1_score(y_test,y_pred,average='weighted')
print("The f1_macro value for the best parameters {}:".format(grid.best_params_),fs)

from sklearn.metrics import roc_auc_score
roc_auc_score=(y_test,grid.predict_proba(x_test)[:1])
roc_auc_score

[[58 30]
 [17 29]]
              precision    recall  f1-score   support

           0       0.77      0.66      0.71        88
           1       0.49      0.63      0.55        46

    accuracy                           0.65       134
   macro avg       0.63      0.64      0.63       134
weighted avg       0.68      0.65      0.66       134

The f1_macro value for the best parameters {'alpha': 0.5}: 0.656979781199175


(376    0
 16     1
 365    1
 82     0
 107    0
       ..
 251    0
 201    0
 386    1
 68     0
 193    0
 Name: Purchased, Length: 134, dtype: int64,
 array([[4.61731066e-248, 1.00000000e+000]]))

In [37]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import CategoricalNB
param_grid = { 'alpha': [0.1, 0.5, 1.0, 1.5, 2.0]
   }
grid=GridSearchCV(CategoricalNB(),param_grid,refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


C:\Users\Hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan]
  warnings.warn(


GridSearchCV(estimator=CategoricalNB(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 1.0, 1.5, 2.0]},
             scoring='f1_weighted', verbose=3)

In [38]:
re=grid.cv_results_
y_pred=grid.predict(x_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)

from sklearn.metrics import classification_report
cr=classification_report(y_test,y_pred)
print(cr)

from sklearn.metrics import f1_score
fs=f1_score(y_test,y_pred,average='weighted')
print("The f1_macro value for the best parameters {}:".format(grid.best_params_),fs)

from sklearn.metrics import roc_auc_score
roc_auc_score=(y_test,grid.predict_proba(x_test)[:1])
roc_auc_score

IndexError: index 15815236 is out of bounds for axis 1 with size 15814554

In [28]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import ComplementNB
param_grid = { 'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],
              'norm': [True, False]
   }
grid=GridSearchCV(ComplementNB(),param_grid,refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


GridSearchCV(estimator=ComplementNB(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],
                         'norm': [True, False]},
             scoring='f1_weighted', verbose=3)

In [29]:
re=grid.cv_results_
y_pred=grid.predict(x_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)

from sklearn.metrics import classification_report
cr=classification_report(y_test,y_pred)
print(cr)

from sklearn.metrics import f1_score
fs=f1_score(y_test,y_pred,average='weighted')
print("The f1_macro value for the best parameters {}:".format(grid.best_params_),fs)

from sklearn.metrics import roc_auc_score
roc_auc_score=(y_test,grid.predict_proba(x_test)[:1])
roc_auc_score

[[58 30]
 [17 29]]
              precision    recall  f1-score   support

           0       0.77      0.66      0.71        88
           1       0.49      0.63      0.55        46

    accuracy                           0.65       134
   macro avg       0.63      0.64      0.63       134
weighted avg       0.68      0.65      0.66       134

The f1_macro value for the best parameters {'alpha': 0.1, 'norm': False}: 0.656979781199175


(376    0
 16     1
 365    1
 82     0
 107    0
       ..
 251    0
 201    0
 386    1
 68     0
 193    0
 Name: Purchased, Length: 134, dtype: int64,
 array([[2.63986204e-248, 1.00000000e+000]]))